## Hypotheses tests

### Supplementary material for Fischer et al "The salt-and-pepper pattern in mouse blastocysts is compatible with signalling beyond the nearest neighbours."

In [2]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
import json
import os
import pandas as pd

Define functions

In [3]:
def highlight_rows(x):
    sig=0.05
    numTests=22
    if x.TTest>sig/numTests and x.Welch>sig/numTests and x.MannWhitney>sig/numTests:
        return['background-color: pink']*4
    elif x.TTest>sig/numTests or x.Welch>sig/numTests or x.MannWhitney>sig/numTests:
        return['background-color: lightblue']*4
    else:
        return['background-color: white']*4

In [4]:
def highlight_rowsRB(x):
    sig=0.01
    numTests=5
    if x.TTest>sig/numTests and x.Welch>sig/numTests and x.MannWhitney>sig/numTests:
        return['background-color: pink']*4
    elif x.TTest>sig/numTests or x.Welch>sig/numTests or x.MannWhitney>sig/numTests:
        return['background-color: lightblue']*4
    else:
        return['background-color: white']*4

In [5]:
def highlight_rowsLC(x):
    sig=0.05
    numTests=10
    if x.TTest>sig/numTests and x.Welch>sig/numTests and x.MannWhitney>sig/numTests:
        return['background-color: pink']*4
    elif x.TTest>sig/numTests or x.Welch>sig/numTests or x.MannWhitney>sig/numTests:
        return['background-color: lightblue']*4
    else:
        return['background-color: white']*4

In [6]:
def highlight_rowsExp(x):
    sig=0.01
    numTests=3
    if x.TTest>sig/numTests and x.Welch>sig/numTests and x.MannWhitney>sig/numTests:
        return['background-color: pink']*4
    elif x.TTest>sig/numTests or x.Welch>sig/numTests or x.MannWhitney>sig/numTests:
        return['background-color: lightblue']*4
    else:
        return['background-color: white']*4

In [7]:
def calculateExpTestValues(resultList,i,moranValuesByStage,expression,stageIndex):
    resultT=stats.ttest_ind([embryo[expression] for embryo in moranValuesByStage[i][1:] 
                                                  if embryo[expression] != 'only one cell type' ],
                            [embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                  if embryo[expression] != 'only one cell type' ])
    resultWelch=stats.ttest_ind([embryo[expression] for embryo in moranValuesByStage[i][1:] 
                                                  if embryo[expression] != 'only one cell type' ],
                                [embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                      if embryo[expression] != 'only one cell type' ], equal_var=False)
    resultMW=stats.mannwhitneyu([embryo[expression] for embryo in moranValuesByStage[i][1:] 
                                                  if embryo[expression] != 'only one cell type' ],
                                [embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                            if embryo[expression] != 'only one cell type' ])
    resultList.append({"label":moranValuesByStage[i][0],"TTest":round(resultT.pvalue,5),"Welch":round(resultWelch.pvalue,5),
                        "MannWhitney":round(resultMW.pvalue,5)})
    return resultList

In [8]:
def calculateTestValues(resultList,i,data,expression,stageIndex,moranValuesByStage):
    resultT=stats.ttest_ind(data[i][1:],[embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                  if embryo[expression] != 'only one cell type' ])
    resultWelch=stats.ttest_ind(data[i][1:],[embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                      if embryo[expression] != 'only one cell type' ], equal_var=False)
    resultMW=stats.mannwhitneyu(data[i][1:],[embryo[expression] for embryo in moranValuesByStage[stageIndex][1:] 
                                                            if embryo[expression] != 'only one cell type' ])
    resultList.append({"label":data[i][0],"TTest":round(resultT.pvalue,5),"Welch":round(resultWelch.pvalue,5),
                        "MannWhitney":round(resultMW.pvalue,5)})
    return resultList

## Experimental data

Import  data from Mathematica for Moran index for experimental data

In [9]:
pathExpData=os.path.join(os. getcwd(),"ResultsAll","expDataForWelchTest.json")

with open(pathExpData, "r") as file:
    moranValuesByStage = json.load(file)

Perform hypothesis testing

In [10]:
resultsListNEarly=[]
resultsListNMid=[]
resultsListNLate=[]

resultsListG6Early=[]
resultsListG6Mid=[]
resultsListG6Late=[]


In [11]:
for i in range(len(moranValuesByStage)):
    #NANOG 
    resultsListNEarly=calculateExpTestValues(resultsListNEarly,i,moranValuesByStage,'N+-Moran',0)
    resultsListNMid=calculateExpTestValues(resultsListNMid,i,moranValuesByStage,'N+-Moran',1)
    resultsListNLate=calculateExpTestValues(resultsListNLate,i,moranValuesByStage,'N+-Moran',2)
    
    #GATA6 
    resultsListG6Early=calculateExpTestValues(resultsListG6Early,i,moranValuesByStage,'G6+-Moran',0)
    resultsListG6Mid=calculateExpTestValues(resultsListG6Mid,i,moranValuesByStage,'G6+-Moran',1)
    resultsListG6Late=calculateExpTestValues(resultsListG6Late,i,moranValuesByStage,'G6+-Moran',2)

Display results

In [12]:
0.01/3

0.0033333333333333335

In [13]:
data=pd.DataFrame(resultsListNEarly)
display(data.style.apply(highlight_rowsExp, axis = 1))

,label,TTest,Welch,MannWhitney
0,early,1.000000,1.000000,1.000000
1,mid,0.244050,0.247780,0.488700
2,late,0.000000,0.000000,0.000000


In [14]:
data=pd.DataFrame(resultsListNMid)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,early,0.244050,0.247780,0.488700
1,mid,1.000000,1.000000,1.000000
2,late,0.000000,0.000000,0.000000


In [15]:
data=pd.DataFrame(resultsListG6Early)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,early,1.000000,1.000000,1.000000
1,mid,0.040870,0.041400,0.079200
2,late,0.000000,0.000000,0.000000


In [16]:
data=pd.DataFrame(resultsListG6Mid)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,early,0.040870,0.041400,0.079200
1,mid,1.000000,1.000000,1.000000
2,late,0.000000,0.000000,0.000000


## Local clustering scan

Import  data from Mathematica for Moran index for rule-based models and experimental data

In [27]:
pathLCModelData=os.path.join(os. getcwd(),"ResultsAll","localClustDataForWelchTest.json")

with open(pathLCModelData, "r") as file:
    plotDataLC = json.load(file)

In [28]:
pathExpData=os.path.join(os. getcwd(),"ResultsAll","expDataForWelchTest.json")

with open(pathExpData, "r") as file:
    moranValuesByStage = json.load(file)

Perform hypothesis testing

In [29]:
resultsListNEarly=[]
resultsListNMid=[]
resultsListNLate=[]

resultsListG6Early=[]
resultsListG6Mid=[]
resultsListG6Late=[]

for i in range(len(plotDataLC)):
    #NANOG 
    resultsListNEarly=calculateTestValues(resultsListNEarly,i,plotDataLC,'N+-Moran',0,moranValuesByStage)
    resultsListNMid=calculateTestValues(resultsListNMid,i,plotDataLC,'N+-Moran',1,moranValuesByStage)
    resultsListNLate=calculateTestValues(resultsListNLate,i,plotDataLC,'N+-Moran',2,moranValuesByStage)
    
    #GATA6 
    resultsListG6Early=calculateTestValues(resultsListG6Early,i,plotDataLC,'G6+-Moran',0,moranValuesByStage)
    resultsListG6Mid=calculateTestValues(resultsListG6Mid,i,plotDataLC,'G6+-Moran',1,moranValuesByStage)
    resultsListG6Late=calculateTestValues(resultsListG6Late,i,plotDataLC,'G6+-Moran',2,moranValuesByStage)

Display results

In [30]:
0.05/11

0.004545454545454546

In [31]:
data=pd.DataFrame(resultsListNEarly)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,0.000000,0.000000,0.000000,0.000000
1,0.100000,0.000000,0.000000,0.000000
2,0.200000,0.000000,0.000000,0.000000
3,0.300000,0.000000,0.000000,0.000000
4,0.400000,0.000000,0.000000,0.000000
5,0.500000,0.000000,0.000000,0.000000
6,0.550000,0.155200,0.136510,0.138710
7,0.600000,0.238950,0.207910,0.231460
8,0.700000,0.000000,0.000000,0.000000
9,0.800000,0.000000,0.000000,0.000000


In [32]:
data=pd.DataFrame(resultsListNMid)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,0.000000,0.000000,0.000000,0.000000
1,0.100000,0.000000,0.000000,0.000000
2,0.200000,0.000000,0.000000,0.000000
3,0.300000,0.000000,0.000000,0.000000
4,0.400000,0.000000,0.000000,0.000000
5,0.500000,0.000000,0.000000,0.000000
6,0.550000,0.006230,0.007060,0.022180
7,0.600000,0.694500,0.690810,0.905930
8,0.700000,0.000000,0.000000,0.000000
9,0.800000,0.000000,0.000000,0.000000


In [34]:
data=pd.DataFrame(resultsListG6Early)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,0.000000,0.000000,0.000000,0.000000
1,0.100000,0.000000,0.000000,0.000000
2,0.200000,0.000000,0.000000,0.000000
3,0.300000,0.000000,0.000000,0.000000
4,0.400000,0.000000,0.000000,0.000000
5,0.500000,0.103930,0.100990,0.089750
6,0.550000,0.027310,0.016660,0.027530
7,0.600000,0.000000,0.000000,0.000000
8,0.700000,0.000000,0.000000,0.000000
9,0.800000,0.000000,0.000000,0.000000


In [35]:
data=pd.DataFrame(resultsListG6Mid)
display(data.style.apply(highlight_rowsLC, axis = 1))

,label,TTest,Welch,MannWhitney
0,0.000000,0.000000,0.000000,0.000000
1,0.100000,0.000000,0.000000,0.000000
2,0.200000,0.000000,0.000000,0.000000
3,0.300000,0.000000,0.000000,0.000000
4,0.400000,0.000000,0.000000,0.000000
5,0.500000,0.000050,0.000070,0.000220
6,0.550000,0.625520,0.599760,0.728620
7,0.600000,0.083930,0.058580,0.054520
8,0.700000,0.000000,0.000000,0.000000
9,0.800000,0.000000,0.000000,0.000000


## Rule-based models

Import  data from Mathematica for Moran index for rule-based models and experimental data

In [34]:
pathRBModelData=os.path.join(os. getcwd(),"ResultsAll","ruleBasedModeldataForWelchTest.json")

with open(pathRBModelData, "r") as file:
    plotData = json.load(file)

In [35]:
pathExpData=os.path.join(os. getcwd(),"ResultsAll","expDataForWelchTest.json")

with open(pathExpData, "r") as file:
    moranValuesByStage = json.load(file)

Perform hypothesis testing

In [36]:
resultsListNEarly=[]
resultsListNMid=[]
resultsListNLate=[]

resultsListG6Early=[]
resultsListG6Mid=[]
resultsListG6Late=[]

for i in range(len(plotData)):
    #NANOG 
    resultsListNEarly=calculateTestValues(resultsListNEarly,i,plotData,'N+-Moran',0,moranValuesByStage)
    resultsListNMid=calculateTestValues(resultsListNMid,i,plotData,'N+-Moran',1,moranValuesByStage)
    resultsListNLate=calculateTestValues(resultsListNLate,i,plotData,'N+-Moran',2,moranValuesByStage)
    
    #GATA6 
    resultsListG6Early=calculateTestValues(resultsListG6Early,i,plotData,'G6+-Moran',0,moranValuesByStage)
    resultsListG6Mid=calculateTestValues(resultsListG6Mid,i,plotData,'G6+-Moran',1,moranValuesByStage)
    resultsListG6Late=calculateTestValues(resultsListG6Late,i,plotData,'G6+-Moran',2,moranValuesByStage)

Display results

In [42]:
0.01/5

0.002

In [46]:
data=pd.DataFrame(resultsListNEarly)
display(data.style.apply(highlight_rowsRB, axis = 1))

,label,TTest,Welch,MannWhitney
0,PeriodTwo,0.000000,0.000000,0.000000
1,Alternating,0.000000,0.000000,0.000000
2,LocalCluster055,0.155200,0.136510,0.138710
3,RandomNANOG,0.000000,0.000000,0.000000
4,RandomGATA6,0.000000,0.000000,0.000000


In [47]:
data=pd.DataFrame(resultsListNMid)
display(data.style.apply(highlight_rowsRB, axis = 1))

,label,TTest,Welch,MannWhitney
0,PeriodTwo,0.000000,0.000000,0.000000
1,Alternating,0.000000,0.000000,0.000000
2,LocalCluster055,0.006230,0.007060,0.022180
3,RandomNANOG,0.000000,0.000000,0.000000
4,RandomGATA6,0.000000,0.000000,0.000000


In [48]:
data=pd.DataFrame(resultsListG6Early)
display(data.style.apply(highlight_rowsRB, axis = 1))

,label,TTest,Welch,MannWhitney
0,PeriodTwo,0.000000,0.000000,0.000000
1,Alternating,0.000000,0.000000,0.000000
2,LocalCluster055,0.027310,0.016660,0.027530
3,RandomNANOG,0.301030,0.324960,0.375690
4,RandomGATA6,0.314580,0.342370,0.285130


In [49]:
data=pd.DataFrame(resultsListG6Mid)
display(data.style.apply(highlight_rowsRB, axis = 1))

,label,TTest,Welch,MannWhitney
0,PeriodTwo,0.000000,0.000000,0.000000
1,Alternating,0.000000,0.000000,0.000000
2,LocalCluster055,0.625520,0.599760,0.728620
3,RandomNANOG,0.000210,0.000610,0.002200
4,RandomGATA6,0.000210,0.000680,0.001380


## Signalling models

Import  data from Mathematica for Moran index for signalling models and experimental data

In [22]:
pathSigModelData=os.path.join(os. getcwd(),"ResultsAll","sigModeldataForWelchTest.json")

with open(pathSigModelData, "r") as file:
    plotDataAllEM = json.load(file)

In [23]:
pathExpData=os.path.join(os. getcwd(),"ResultsAll","expDataForWelchTest.json")

with open(pathExpData, "r") as file:
    moranValuesByStage = json.load(file)

Perform hypothesis tests

In [25]:
resultsListNEarly=[]
resultsListNMid=[]
resultsListNLate=[]

resultsListG6Early=[]
resultsListG6Mid=[]
resultsListG6Late=[]

for i in range(len(plotDataAllEM)):
    #NANOG 
    resultsListNEarly=calculateTestValues(resultsListNEarly,i,plotDataAllEM,'N+-Moran',0,moranValuesByStage)
    resultsListNMid=calculateTestValues(resultsListNMid,i,plotDataAllEM,'N+-Moran',1,moranValuesByStage)
    resultsListNLate=calculateTestValues(resultsListNLate,i,plotDataAllEM,'N+-Moran',2,moranValuesByStage)
    
    #GATA6 
    resultsListG6Early=calculateTestValues(resultsListG6Early,i,plotDataAllEM,'G6+-Moran',0,moranValuesByStage)
    resultsListG6Mid=calculateTestValues(resultsListG6Mid,i,plotDataAllEM,'G6+-Moran',1,moranValuesByStage)
    resultsListG6Late=calculateTestValues(resultsListG6Late,i,plotDataAllEM,'G6+-Moran',2,moranValuesByStage)

Display results

In [26]:
0.05/22

0.002272727272727273

In [28]:
data=pd.DataFrame(resultsListNEarly)
display(data.style.apply(highlight_rows, axis = 1))

,label,TTest,Welch,MannWhitney
0,NN,0.000000,0.000000,0.000000
1,NNCD,0.000000,0.000000,0.000000
2,0.05,0.000000,0.000000,0.000000
3,0.05 CD,0.462300,0.447410,0.819150
4,0.15,0.000000,0.000000,0.000000
5,0.15 CD,0.838340,0.833540,0.743620
6,0.25,0.000000,0.000000,0.000000
7,0.25 CD,0.643240,0.639990,0.494480
8,0.35,0.000000,0.000000,0.000000
9,0.35 CD,0.172260,0.169550,0.072460


In [27]:
data=pd.DataFrame(resultsListNMid)
display(data.style.apply(highlight_rows, axis = 1))

,label,TTest,Welch,MannWhitney
0,NN,0.000000,0.000000,0.000000
1,NNCD,0.000000,0.000000,0.000000
2,0.05,0.000000,0.000000,0.000000
3,0.05 CD,0.068420,0.064760,0.374010
4,0.15,0.000000,0.000000,0.000000
5,0.15 CD,0.157540,0.158530,0.751290
6,0.25,0.000000,0.000000,0.000000
7,0.25 CD,0.417990,0.425620,0.924840
8,0.35,0.000000,0.000000,0.000000
9,0.35 CD,0.999400,0.999410,0.360690


In [29]:
data=pd.DataFrame(resultsListG6Early)
display(data.style.apply(highlight_rows, axis = 1))

,label,TTest,Welch,MannWhitney
0,NN,0.000000,0.000000,0.000000
1,NNCD,0.000000,0.000000,0.000000
2,0.05,0.000000,0.000000,0.000000
3,0.05 CD,0.037900,0.033820,0.010550
4,0.15,0.000000,0.000000,0.000000
5,0.15 CD,0.005070,0.004260,0.000870
6,0.25,0.000000,0.000000,0.000000
7,0.25 CD,0.000490,0.000460,0.000180
8,0.35,0.000000,0.000000,0.000000
9,0.35 CD,0.000010,0.000010,0.000000


In [30]:
data=pd.DataFrame(resultsListG6Mid)
display(data.style.apply(highlight_rows, axis = 1))

,label,TTest,Welch,MannWhitney
0,NN,0.000000,0.000000,0.000000
1,NNCD,0.000000,0.000000,0.000000
2,0.05,0.000000,0.000000,0.000000
3,0.05 CD,0.930680,0.930530,0.562210
4,0.15,0.000000,0.000000,0.000000
5,0.15 CD,0.642770,0.646560,0.246530
6,0.25,0.000000,0.000000,0.000000
7,0.25 CD,0.273350,0.281540,0.120580
8,0.35,0.000000,0.000000,0.000000
9,0.35 CD,0.050680,0.055700,0.014360
